# Generate and Execute a Bash Script

This notebook builds on the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe. Here, the generated Bash code is executed as a system command.

> **WARNING:** This recipe executes code generated by language models. The generated code may delete or modify files on your system. Use with caution!

See the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe for instructions on installing [Ollama](https://ollama.com/), which you will need, and for information on concepts like _system prompts_ and the differences between commands on different operating systems.

In [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb), we used a Python helper function to determine the host operating system, MacOS, Linux, etc. We used this information in the prompts to encourage the model to generate shell commands that work on the host system, because some shell commands differ between operating systems. However, the output may still contain commands that are not supported by your operating system. Because this notebook attempts to run the generated commands, you may see failures if an incorrect command syntax was generated.

Let's begin by repeating some of the code we saw in [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb), like `pip` installing the Ollama Python API, if needed, defining some data, like the `examples`, and one of the helper functions, `os_name()`.

In [ ]:
!pip install ollama

In [ ]:
import ollama
import os
import platform

In [ ]:
def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS. 
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    return name_map.get(os_name, os_name)  

In [ ]:
stat_flags = '-c "%y %n" {}'
if os_name() == 'MacOS':
    stat_flags = '-f "%m %N" {}'
print(f"The 'stat' flags for my OS \'{os_name()}\' are \'{stat_flags}\'")

In [ ]:
examples = f"""
Question:
Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.
Answer:
find . -name '*.js' | grep -v excludeddir

Question:
Dump \"a0b\" as hexadecimal bytes
Answer:
printf \"a0b\" | od -tx1

Question:
create a tar ball of all pdf files in the current folder and any subdirectories.
Answer:
find . -name '*.pdf' | xargs tar czvf pdf.tar

Question:
Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items
Answer:
find . -maxdepth 1 -exec stat {stat_flags} \; | sort -n -r | tail -n 7

Question:
find all the empty directories in and under the current directory.
Answer:
find . -type d -empty

"""

## Writing Prompts for Direct Execution

Rather than work through the three successive ways to invoke generation using Ollama, we will just use the `chat()` helper function we used in the previous notebook. 

But first, we want to modify the _system prompt_ we used in the previous notebook to work better for our purposes.

Writing prompts is an art. Recall in [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb), our output was usually Markdown with quoted sections of shell code and commentary explaining how it worked. Here, we just want code output that we execute without editing. Here are a few tips on writing prompts for our purposes here:

> **TIPS:**
>
> 1. Rather than use a question ("How do I ...?") in a prompt, provide a directive ("Write a script that ..."). This helps prevent the model from generating dialogue around the code.
> 2. Add instructions to the system prompt like this: "You are a helpful software engineer. You write clear, concise, well-commented code. You only print valid code. You don't print any commentary about the code nor markdown syntax to wrap the code." 

So here is our new system prompt:

In [ ]:
default_system_prompt = f"""
    You are a helpful software engineer. You write clear, concise, 
    well-commented code. You only print valid code. You don't print 
    any commentary about the code nor markdown syntax to wrap the code.
    You make sure you only generate code that is {os_name()}-compatible!
    """

Here is the same `chat()` helper from before.

In [ ]:
def chat(prompt: str, 
         system_prompt:str = default_system_prompt,
         examples: str = examples, 
         model: str ='granite-code:8b') -> str:
    user_prompt = f"""
        {examples}
        Question:
        {prompt}
        Answer:""" 
    
    response = ollama.chat(model=model, messages=[
      {
        'role':'system',
        'content': system_prompt  
      },
      {
        'role': 'user',
        'content': user_prompt
      },
    ])

    result = response['message']['content']
    print(result)
    return result

In [ ]:
bash_code1 = chat("""
    Write a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

Compare this output to what you got in the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe. Is this output a valid script and nothing else? Or, is there extra commentary and Markdown formatting? If you got this extra, undesirable output, try running the cell again. Does modifying the prompt or system prompt help?

Now we can attempt to execute the script! There is no need for an additional helper function:

In [ ]:
os.system(bash_code1)

Let's try another one.

In [ ]:
bash_code2 = chat("""Write a bash script to recursively find Jupyter notebooks
in the parent directory and print their paths.""")

In [ ]:
os.system(bash_code2)